In [1]:
include("./trajopt/utils.jl")
# include("./trajopt/dynamics.jl")
# include("./funlopt/funl_dynamics.jl")
# include("./funlopt/funl_utils.jl")
# include("./funlopt/funl_synthesis.jl")
# include("./funlopt/funl_constraint.jl")
# include("./trajopt/scaling.jl")
using Plots
using LinearAlgebra
# using Random

In [2]:
using MuJoCo
import MuJoCo: step! as mj_step
using Interpolations
using MatrixEquations

In [3]:
# model = load_model("./xmls/skydio_x2/scene.xml")
model = load_model("./xmls/my_quadrotor.xml")
data = init_data(model)

MuJoCo Data object

In [4]:
# init_visualiser()
# visualise!(model, data)

In [5]:
forward!(model, data)
data.qpos

7×1 transpose(::UnsafeArrays.UnsafeArray{Float64, 2}) with eltype Float64:
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0

In [6]:
H=mj_zeros(model.nv, model.nv)
L = mj_fullM(model,H , data.qM)
display(H)
J = H[end-2:end,end-2:end]
display(J)
m = 1.325
g = 9.81

6×6 transpose(::Matrix{Float64}) with eltype Float64:
 1.325   0.0    0.0     0.0      0.05     0.0
 0.0     1.325  0.0    -0.05     0.0      0.0
 0.0     0.0    1.325   0.0      0.0      0.0
 0.0    -0.05   0.0     0.03843  0.0      0.0
 0.05    0.0    0.0     0.0      0.02719  0.0
 0.0     0.0    0.0     0.0      0.0      0.060528

3×3 Matrix{Float64}:
 0.03843  0.0      0.0
 0.0      0.02719  0.0
 0.0      0.0      0.060528

9.81

# Geometric control

In [7]:
frequency = 2*pi/5
radius = 5
radius * frequency

dt_control = model.opt.timestep
sim_time = 30.0
t_ctrl = range(0,sim_time,step=dt_control)

x_path = radius .* cos.(frequency.*t_ctrl)
y_path = radius .* sin.(frequency.*t_ctrl);


In [8]:
function hat_map(v::Vector)
    return [0.0 -v[3] v[2];
            v[3] 0.0 -v[1];
            -v[2] v[1] 0.0]
end
function vee_map(skew::Matrix)
    vec = 1/2 .* [skew[3,2]-skew[2,3];skew[1,3]-skew[3,1];skew[2,1]-skew[1,2]]
    return vec
end
function dir_cosine(q)
    C_B_I = zeros(3,3)
    C_B_I[1,1] = 1 - 2 * (q[3]^2 + q[4]^2)
    C_B_I[1,2] = 2 * (q[2] * q[3] + q[1] * q[4])
    C_B_I[1,3] = 2 * (q[2] * q[4] - q[1] * q[3])
    C_B_I[2,1] = 2 * (q[2] * q[3] - q[1] * q[4]) 
    C_B_I[2,2] = 1 - 2 * (q[2]^2 + q[4]^2) 
    C_B_I[2,3] = 2 * (q[3] * q[4] + q[1] * q[2])
    C_B_I[3,1] = 2 * (q[2] * q[4] + q[1] * q[3])
    C_B_I[3,2] = 2 * (q[3] * q[4] - q[1] * q[2])
    C_B_I[3,3] =  1 - 2 * (q[2]^2 + q[3]^2)
    return C_B_I'
end
function geometric_control(t::Float64,x::Vector)
    rx = x[1]
    ry = x[2]
    rz = x[3]

    quat = x[4:7]

    vx = x[8]
    vy = x[9]
    vz = x[10]

    p = x[11]
    q = x[12]
    r = x[13]
    omega = [p;q;r]

    # R
    # R = get_R([phi,theta,psi])
    R = dir_cosine(quat)

    # desired position (hard coding)
    b = 1.0
    zdes = 5.0 * (1 - exp(- b*t)) + 1
    zdes_dot = 5 * b * exp(-b*t)
    zdes_ddot = 5 * b * (-b) * exp(-b * t)

    
    xdes = radius * cos(frequency*t)
    xdes_dot = - radius * frequency * sin(frequency*t)
    xdes_ddot = - radius * frequency^2 * cos(frequency*t)
    ydes = radius * sin(frequency*t)
    ydes_dot = radius * frequency * cos(frequency*t)
    ydes_ddot = - radius * frequency^2 * sin(frequency*t)
    # xdes = 0.0
    # xdes_dot = 0.0
    # xdes_ddot = 0.0
    # ydes = 0.0
    # ydes_dot = 0.0
    # ydes_ddot = 0.0

    psi_des = deg2rad(0)
    psi_des_dot = 0.0

    # errors
    ex = [rx;ry;rz] - [xdes;ydes;zdes]
    ev = [vx;vy;vz] - [xdes_dot;ydes_dot;zdes_dot]

    # gains
    kR = 30 
    komega = 5

    kx = 5
    kv = 2

    # total thrust
    xddot_des = [xdes_ddot;ydes_ddot;zdes_ddot]
    Fd = - kx * ex - kv * ev + m.*g.*[0;0;1] + m * xddot_des
    Fz = dot(Fd,R * [0;0;1])

    # desired attitudes
    if norm(Fd) < 1e-8
        @error("Fd is too small")
    end
    b3d = Fd ./ norm(Fd)
    b1d = [cos(psi_des);sin(psi_des);0]
    b2d = cross(b3d,b1d)
    b2d = b2d ./ norm(b2d)
    Rd = zeros(3,3)
    Rd[:,1] = cross(b2d,b3d)
    Rd[:,2] = b2d
    Rd[:,3] = b3d
    Rd_dot = dir_cosine([1;0;0;0])
    omega_des = zeros(3)
    omega_dot_des = zeros(3)

    # errors
    eR = 0.5 * vee_map(Rd'*R - R'*Rd)
    e_omega = omega - R'*Rd*omega_des

    M = (-kR .* eR - komega .* e_omega) + cross(omega,J*omega) - J*(hat_map(omega)*R'*Rd*omega_des - R'*Rd*omega_dot_des)

    return Fz,M
end
dx = 0.14
dy = 0.18
c = 0.0201
N = [1 1 1 1;-dy dy dy -dy;dx dx -dx -dx;-c c -c c]
N_inv = inv(N)
function actuator_allocation(Fz::Float64,M::Vector)
    return N_inv * [Fz;M]
end

actuator_allocation (generic function with 1 method)

In [9]:
reset!(model,data)
ctrl_states = zeros(13, length(t_ctrl))
for (idx,t_) in enumerate(t_ctrl)
    @assert(isapprox(t_, data.time))
    ctrl_states[:,idx] = get_physics_state(model, data)
    Fz, M = geometric_control(t_,ctrl_states[:,idx])
    println(Fz,M)
    data.ctrl .= actuator_allocation(Fz,M)
    mj_step(model, data)
end

16.37325[-16.599047165457787, 15.490360795718981, -5.259176694270006]
15.963366037074957[-6.99741343815302, 4.1523761938224695, -4.538976126290353]
15.642250601948032[-3.0872110136626905, -0.8247527285185913, -3.7602373261934563]
15.469126747907776[-2.604453278421108, -1.9252460414469628, -3.0857812452355438]
15.408925311030895[-2.7126702060093337, -2.349872054770241, -2.4981240519275887]
15.442510449862924[-2.921351473590507, -2.6482970776170114, -1.9823832686368554]
15.555558739723912[-3.1439966387785203, -2.9147901599549377, -1.5273558540568881]
15.736205240321283[-3.364277386582016, -3.1655344314169174, -1.1241957563307003]
15.972184087849794[-3.54529509934558, -3.364501280100371, -0.7684494002123062]
16.217161423937778[-3.1015093972476664, -2.842124801448676, -0.499636365333161]
16.4823162753482[-2.3088955120966017, -2.14912415464803, -0.26869782582395063]
16.771932633461326[-1.3574648605743158, -1.4042253545025187, -0.05715336796519252]
17.087594605275566[-0.342545968303794, -0.6

In [ ]:
plot(ctrl_states[1,:],ctrl_states[2,:],title="position")
plot!(x_path,y_path,color=:red)
xlabel!("X (m)")
ylabel!("Y (m)")

In [ ]:
pz = plot(t_ctrl,ctrl_states[3,:],title="altitude")
pz = ylabel!("Up")

In [ ]:
init_visualiser()
visualise!(model, data, trajectories = ctrl_states)